In [26]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tslearn.clustering import TimeSeriesKMeans
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, LSTM, Input, Flatten, Reshape
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
import numpy as np
import os
import pandas as pd
import pywt

In [27]:
epoch_num = 5
sequence_length = 6  # Number of time steps to consider

# Define file and directory names
file_name = '0_61938.csv'
data_dir = 'ohlc_data'
# parent_dir = 'forexPredictor'
# repo = 'Repos_git'
# repo_dir = 'Make_Money_with_Tensorflow_2.0'
# Get the absolute base directory dynamically
base_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))  # Move up one level

# Construct the full file path in an OS-independent way
# data_path = os.path.join(base_dir, repo, repo_dir, parent_dir, data_dir, file_name)
data_path = os.path.join(base_dir, data_dir, file_name)

# Check if the file exists before using it
if os.path.exists(data_path):
    print(f"✅ File found: {data_path}")
else:
    print(f"❌ Error: File not found at {data_path}")


ucols=['Open', 'High', 'Low', 'Close']
data = pd.read_csv(data_path, usecols=ucols)
data.reset_index(drop=True, inplace=True)
data.head()


✅ File found: D:\Repos_git\Make_Money_with_Tensorflow_2.0\forexPredictor\ohlc_data\0_61938.csv


,Open,High,Low,Close
0,1.20997,1.21089,1.20966,1.20999
1,1.20481,1.20569,1.20479,1.20538
2,1.20537,1.20574,1.20341,1.20553
3,1.20556,1.20689,1.20442,1.20469
4,1.20468,1.20599,1.20380,1.20573


In [36]:
# Normalize data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
data = scaler.fit_transform(df)

# Reshape for LSTM input
data = data.reshape((data.shape[0], data.shape[1], 1))

In [70]:
def apply_dwt_flat(data):
    coeffs_level1 = [pywt.dwt(sample.flatten(), 'haar') for sample in data]  # First-level DWT
    cA1, cD1 = zip(*coeffs_level1)  # Approximation and detail coefficients (2 features)

    coeffs_level2 = [pywt.dwt(cA, 'haar') for cA in cA1]  # Second-level DWT on Approximation
    cA2, cD2 = zip(*coeffs_level2)  # Now we have 4 features

    transformed_data = np.array(list(zip(cA2, cD2, cD1)))  # Combine all 4 coefficients
    return transformed_data.reshape(transformed_data.shape[0], transformed_data.shape[1], 1)  # Ensure shape (713, 4, 1)

def apply_dwt(data):
    coeffs = [pywt.dwt(sample, 'haar') for sample in data]
    cA, cD = zip(*coeffs)  # Approximation and Detail coefficients
    return np.array(cA)  # Using only approximation coefficients

# Define Autoencoder with LSTM for better feature extraction
def create_autoencoder(input_shape):
    input_layer = Input(shape=input_shape)
    x = LSTM(64, return_sequences=True)(input_layer)
    x = LSTM(32, return_sequences=False)(x)
    encoded = Dense(16, activation='relu')(x)

    decoded = Dense(32, activation='relu')(encoded)
    decoded = Dense(input_shape[0], activation='sigmoid')(decoded)

    autoencoder = Model(input_layer, decoded)
    encoder = Model(input_layer, encoded)

    autoencoder.compile(optimizer='adam', loss='mse')
    return autoencoder, encoder


In [29]:
df = data

In [30]:
# Convert DataFrame to NumPy array
data = df.values



In [38]:
# Apply DWT
transformed_data = apply_dwt(data)

# Train Autoencoder
autoencoder, encoder = create_autoencoder(transformed_data.shape[1:])
autoencoder.fit(transformed_data, transformed_data, epochs=5, batch_size=32, verbose=1)


Epoch 1/5
1936/1936 [==============================] - 9s 3ms/step - loss: 0.0089
Epoch 2/5
1936/1936 [==============================] - 6s 3ms/step - loss: 0.0059
Epoch 3/5
1936/1936 [==============================] - 6s 3ms/step - loss: 0.0059
Epoch 4/5
1936/1936 [==============================] - 6s 3ms/step - loss: 0.0059
Epoch 5/5
1936/1936 [==============================] - 6s 3ms/step - loss: 0.0059


In [53]:
autoencoder.save("encoder_model.h5")

C:\Users\Teacher\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [40]:
# Extract Features
features = encoder.predict(transformed_data)

# Apply TimeSeriesKMeans Clustering
num_clusters = 3  # Buy (2), Sell (1), Hold (0)
tskmeans = TimeSeriesKMeans(n_clusters=num_clusters, metric="dtw", random_state=42)
clusters = tskmeans.fit_predict(features)


1936/1936 [==============================] - 3s 1ms/step


In [41]:
# Convert clusters to one-hot encoding
clusters_one_hot = tf.keras.utils.to_categorical(clusters, num_clusters)

# Split data into train and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(features, clusters_one_hot, test_size=0.2, random_state=42)

# Ensure input shape consistency
input_shape = features.shape[1:]  # Extract feature dimension
classifier_input = Input(shape=input_shape)
classifier_output = Dense(num_clusters, activation="softmax")(classifier_input)

# Create and compile classifier
final_model = Model(classifier_input, classifier_output)
final_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

epoch_num = 5  # Define the number of epochs
final_model.fit(X_train, y_train, epochs=epoch_num, batch_size=5, validation_data=(X_test, y_test))

# Evaluate model on test data
loss, accuracy = final_model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


Epoch 1/5
9910/9910 [==============================] - 9s 890us/step - loss: 0.1636 - accuracy: 0.9511 - val_loss: 0.0713 - val_accuracy: 0.9856
Epoch 2/5
9910/9910 [==============================] - 8s 841us/step - loss: 0.0562 - accuracy: 0.9902 - val_loss: 0.0454 - val_accuracy: 0.9907
Epoch 3/5
9910/9910 [==============================] - 8s 844us/step - loss: 0.0406 - accuracy: 0.9947 - val_loss: 0.0372 - val_accuracy: 0.9879
Epoch 4/5
9910/9910 [==============================] - 9s 873us/step - loss: 0.0334 - accuracy: 0.9956 - val_loss: 0.0310 - val_accuracy: 0.9975
Epoch 5/5
388/388 [==============================] - 0s 661us/step - loss: 0.0279 - accuracy: 0.9964
Test Accuracy: 99.64%


In [42]:
final_model.save("trading_model_dwt_autoEnc.h5")
loaded_model = load_model("trading_model_dwt_autoEnc.h5")


C:\Users\Teacher\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [43]:
# documentation: https://www.mql5.com/en/docs/integration/python_metatrader5

import MetaTrader5 as mt  # pip install MetaTrader5
import pandas as pd  # pip install pandas
from datetime import datetime, timedelta
import os

# start the platform with initialize()
mt.initialize()

# login to Trade Account with login()
# make sure that trade server is enabled in MT5 client terminal

login = 165905041
password = 'iIeElL0176_'
server = 'XMGlobal-MT5 2'

mt.login(login, password, server)



True

In [44]:
# Define symbol and timeframe
symbol = "EURUSD"
timeframe = mt.TIMEFRAME_H1

# Get data for the last 48 hours
end_time = datetime.now()
start_time = end_time - timedelta(hours=1000)

# Retrieve OHLC data
ohlc_data = pd.DataFrame(mt.copy_rates_range(symbol, timeframe, start_time, end_time))

# Convert time column to datetime format
ohlc_data['time'] = pd.to_datetime(ohlc_data['time'], unit='s')


ohlc_data.rename(columns={'open':'Open', 'high':'High', 
                        'low':'Low', 'close':'Close', 'tick_volume':'Volume'}, inplace=True)
columns_to_drop = ['time', 'spread', 'real_volume', 'Volume']
ohlc_data.drop(columns=[col for col in columns_to_drop if col in ohlc_data.columns], inplace=True)



In [45]:
ohlc_data

,Open,High,Low,Close
0,1.03885,1.03925,1.03837,1.03902
1,1.03902,1.03965,1.03871,1.03908
2,1.03909,1.03955,1.03798,1.03811
3,1.03809,1.03854,1.03789,1.03831
4,1.03831,1.03913,1.03815,1.03913
...,...,...,...,...
708,1.07844,1.07875,1.07812,1.07861
709,1.07857,1.07906,1.07803,1.07851
710,1.07851,1.07988,1.07843,1.07983
711,1.07983,1.08019,1.07920,1.07949


In [65]:
df = new_data = ohlc_data

In [66]:
# Load the saved scaler (Assuming it's saved from training to ensure consistency)
# import joblib
# scaler = joblib.load("scaler.pkl")

# Load new data
new_data = new_data[['Open', 'High', 'Low', 'Close']].values  # Extract OHLC values

# Normalize using the same MinMaxScaler used during training
ohlc_data_scaled = scaler.transform(new_data)  # Use transform, NOT fit_transform

# Reshape for LSTM input
data = ohlc_data_scaled.reshape((new_data.shape[0], new_data.shape[1], 1))


C:\Users\Teacher\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [72]:
data.shape

(713, 4, 1)

In [73]:
# Apply DWT
transformed_data = apply_dwt_flat(data)
print("Transformed Data Shape:", transformed_data.shape)


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (713, 3) + inhomogeneous part.

In [58]:


# Load trained encoder and classifier
encoder = tf.keras.models.load_model("encoder_model.h5")
classifier = tf.keras.models.load_model("trading_model_dwt_autoEnc.h5")

# Extract features using the trained encoder
features = encoder.predict(transformed_data)

# Make predictions
predictions = classifier.predict(features)
predicted_classes = np.argmax(predictions, axis=1)  # Convert probabilities to class labels

# Map predictions to labels
labels = {0: "Hold", 1: "Sell", 2: "Buy"}
predicted_labels = [labels[p] for p in predicted_classes]

# Align predictions with the original DataFrame
new_data_df = pd.DataFrame(new_data, columns=['Open', 'High', 'Low', 'Close'])
new_data_df["Prediction"] = predicted_labels  # Add predicted actions to DataFrame

# Save the results
new_data_df.to_csv("predicted_results.csv", index=False)

# Print first 10 rows with predictions
print(new_data_df.head(10))


ValueError: in user code:

    File "C:\Users\Teacher\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 2416, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\Teacher\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 2401, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Teacher\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 2389, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\Teacher\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 2357, in predict_step
        return self(x, training=False)
    File "C:\Users\Teacher\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb